In [ ]:
#import dependancies
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, functions as F
import re
import numpy as np
import us


In [ ]:
#starting variables
startpath ="_Star_Ratings_and_Display_Measures/" #start of star rating path
fallpath ="_Star_Ratings_Fall_Release/"#path for fall ratings
cpath="_Part_C"#path for part c data
dpath="_Part_D"#path for part d data
finalpath ="_Report_Card_Master_Table.xlsx"#last part of star rating path
firstfive = ["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"]#name of first five columns
lasttwo =["Year","Overall"] #name of what will be final 2 columns


In [ ]:
#function to get measure star information
#takes in path of the file
#returns dataframe of the star information
def get_measure_stars(path):
    dfms = pd.read_excel(path,"Measure_Stars",header=2)#df of the raw information
    dfms = dfms.iloc[1:]#remove first line of data
    #loop through the columns after the first five
    for x in range(5,len(dfms.keys())):
        #remove letter number information from column name
        dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
    #loop through the first 5 columns to give them proper names
    for x in range(5):
        #rename the columns
        dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
    #return the data frame
    return dfms

In [ ]:
#function to get Domain star information
#takes in path of the file
#returns dataframe of the star information
def get_domain_stars(path):
    dfds = pd.read_excel(path,"Domain_Stars",header=1)
    return dfds

In [ ]:
#function to get summary star information
#takes in path of the file
#returns dataframe of the star information
def get_summary_rating(path):
    #read in data frame
    dfsr = pd.read_excel(path,"Summary_Rating",header=1)
    #remove the sanction deduction column
    dfsr = dfsr.drop(columns="Sanction Deduction",errors='ignore')
    #retrun dataframe
    return dfsr

In [ ]:
#function to combine the 3 dataframes and do basic cleanup on them used for early years
#takes year of the function
#returns the cleaned dataframe
def get_early(y):
    #build path for files
    fullpath = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
    #get measure stars info
    dfmsf = get_measure_stars(fullpath)
    #get domain star info
    dfdsf = get_domain_stars(fullpath)
    #get summary star info
    dfsrf = get_summary_rating(fullpath)
    #merge measure and domain stars
    dff = pd.merge(dfmsf,dfdsf,on=firstfive,how='left')
    #merge measure, domain, and summary star dataframes
    dff = pd.merge(dff,dfsrf,on=firstfive,how='left')
    #add year column
    dff["Year"] = f"{y}"
    #rename columns
    dff = dff.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    #drop unneeded columns
    dff = dff.drop(columns = "2017 Disaster %",errors='ignore')
    #return dataframe
    return dff

In [ ]:
#function to combine the 3 dataframes and do basic cleanup on them used for later years
#takes year of the function
#returns the cleaned dataframe
def get_late(y):
    #build path for files
    fullpath = f"./Data/{y}{startpath}{y}{finalpath}"
    #get measure stars info
    dfmsc = get_measure_stars(fullpath)
    #get domain star info
    dfdsc = get_domain_stars(fullpath)
    #get summary star info
    dfsrc = get_summary_rating(fullpath)
    #drop disaster columns
    dfsrc = dfsrc.drop(columns=dfsrc.columns[[6,7]],axis = 1)
    #merge measure and domain stars
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    #merge measure, domain, and summary star dataframes
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    #rename columns
    df = df.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    #add year column
    df["Year"] = f"{y}"
    #return dataframe
    return    df

In [ ]:
#function to combine the dataframse for all of the years
#no input
#returns data frame with data from all years
def standardize_data():
    #set first year
    year = 2014
    #set years of data
    years = [x for x in range(2015,2026)]
    #set path for part C 2014        
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{cpath}{finalpath}"
    #get measure star dataframe for part C 2014
    dfmsc = get_measure_stars(fullpath)
    #get domain star dataframe for part C 2014
    dfdsc = get_domain_stars(fullpath)
    #get summary star dataframe for part C 2014
    dfsrc = get_summary_rating(fullpath)
    #merge measure and domain star dataframes for part C 2014
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    #merge measure, domain, and summary star dataframes for part C 2014
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    #set path for part D 2014    
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{dpath}{finalpath}"
    #get measure star dataframe for part D 2014
    dfmsd = get_measure_stars(fullpath)
    #get domain star dataframe for part D 2014
    dfdsd = get_domain_stars(fullpath)
    #get summary star dataframe for part D 2014
    dfsrd = get_summary_rating(fullpath)
    #merge measure and domain star dataframes for part D 2014
    dfd = pd.merge(dfmsd,dfdsd,on=['Contract Number'],how='left')
    #merge measure, domain, and summary star dataframes for part D 2014
    dfd = pd.merge(dfd,dfsrd,on=['Contract Number'],how='left')
    #drop duplicate columns
    dfd = dfd.drop(columns=["Organization Type_x","Contract Name_x","Organization Marketing Name_x","Parent Organization_x","Organization Type_y","Contract Name_y","Organization Marketing Name_y","Parent Organization_y"])
    #merge part C and D data
    df = pd.merge(df,dfd,on=["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization","SNP"],how='left')
    #rename columns
    df = df.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})
    #add year to  dataframes
    df["Year"] = "2014"
    finaldf = df
    #loop through years
    for year in years:
        #choose which merger to call if early call early otherwise call late
        if (year <2020):
            df = get_early(year)
        else:
            df = get_late(year)
        #concatinate the dataframes into one big data frame
        newdf = pd.concat([finaldf,df],axis=0,join='outer')
        #change the dataframe so it can be reused
        finaldf=newdf
    #reset index of dataframe    
    finaldf = finaldf.reset_index(drop=True)
    #return final dataframe
    return finaldf

In [ ]:
#function to clean the dataframe
#takes in a dataframe
#returns cleaned dataframe
def clean_data(combodf1):
    #loop through columns in dataframe
    for x in range(len(combodf1.keys())):
        #change datatypes to string
        combodf1[combodf1.keys()[x]] = combodf1[combodf1.keys()[x]].astype(str)
    #trim various typs of data to remove excess spaces
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*new\\s*to\\s*be\\s*measured\\s*','Plan too new to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*small\\s*to\\s*be\\s*measured\\s*','Plan too small to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*not\\s*required\\s*to\\s*report\\s*measure\\s*','Plan not required to report measure',regex=True)
    combodf1 = combodf1.replace('\\s*No\\s*data\\s*available\\s*','No data available',regex=True)
    combodf1 = combodf1.replace('Nodata available','No data available',regex=True)
    combodf1 = combodf1.replace('\\s*Not\\s*enough\\s*data\\s*available\\s*','Not enough data available',regex=True)
    combodf1 = combodf1.replace('\\s*Benefit\\s*not\\s*offered\\s*by\\s*plan\\s*','Benefit not offered by plan',regex=True)
    #change yes and not to true and false
    combodf1 = combodf1.replace('\\s*Yes\\s*',True,regex=True)    
    combodf1 = combodf1.replace('\\s+No\\s+',False,regex=True)
    combodf1 = combodf1.replace('\\s+No',False,regex=True)
    combodf1 = combodf1.replace('No\\s+',False,regex=True)
    #replace nans that were created by adding columns with 0s
    combodf1 = combodf1.replace('nan','0')
    #return the dataframe
    return combodf1

In [ ]:
#function to be used to create non numeric columns
#takes in a dataframe item
#returns either that item or a string
def add_non_numeric_cols(item):
    #attempt to set type to decimal
    try:
        #if it was able to make a decimal return numeric
        val = float(item)
        return "Numeric"
    except:
        #otherwise return the string
        return item

In [ ]:
#function to clean column of non numeric data
#takes in dataframe item
#returns 0 or the decimal version of item.
def change_to_dec(item):
    #attemp to set type to decimal
    try:
        #if successful return item
        val = float(item)
        return val
    except:
        #otherwise return 0
        return 0

In [ ]:
#function to bucket standardized enrollment
#takes in dataframe item
#returns bucket of the data
def change_enrolled(item):
    #first bucket
    if item <.05:
        return 0
    #second bucket
    elif item < .3:
        return 1
    #third bucket
    elif item < .7:
        return 2
    #final bucket
    else:
        return 3

In [ ]:
#function to add enrollment data to a dataframe
#takes in a dataframe
#returns dataframe combined with enrollment data
def combine_enrollment(newdf1):
    #read in enrollment data
    enrollmentcsv = pd.read_excel("./Data/Enrollment.xlsx")
    #read in state population
    statepop = pd.read_csv("./CSVs/StatePopulations.csv")
    #map state fips and abbreviation
    abfips = us.states.mapping('fips','abbr')
    #change mapping into dataframe
    abfips = pd.DataFrame.from_dict([abfips])
    abfips = abfips.transpose()
    abfips = abfips.reset_index()
    abfips = abfips.rename(columns={"index":"fips",0:"abbr"})
    #merge enrollment and state info
    enrollab = pd.merge(enrollmentcsv,abfips,left_on='State',right_on='abbr')
    enrollab = enrollab.drop(columns="abbr")
    enrollab['fips'] = enrollab['fips'].astype(int)
    #merge enrollement with state population
    enrollabpop = pd.merge(enrollab,statepop,left_on=['fips','Year'],right_on=['state','Year'])
    enrollabpop = enrollabpop.drop(columns=['state'])
    #find average enrollment in a year
    testgroup = pd.DataFrame(enrollabpop.groupby(['Contract Number','State','Year'])['Total Enrollments by State'].mean())
    testgroup = testgroup.reset_index()
    testgroup['Total Enrollments by State'] = testgroup['Total Enrollments by State'].round(0)
    #rename columns
    testgroup = testgroup.rename(columns={"Total Enrollments by State":"Avg Enrollment"})
    #merge average enrollment into enrollment
    enrollwavg = pd.merge(testgroup,enrollabpop,on=['Contract Number', 'State','Year'])
    #standardize enrollment for state population
    enrollwavg["Standardized Enrollment"] = enrollwavg['Avg Enrollment']/enrollwavg['Population Over 65']
    #remove unneccesary columns and drop duplicates that removing columns caused
    enrollwavg1 = enrollwavg.copy()
    enrollwavg1 = enrollwavg1.drop(columns=['Month','Total Enrollments by State'])
    enrollwavg1 = enrollwavg1.drop_duplicates()
    datatomerge = enrollwavg1.groupby(["Contract Number","Year"])['Standardized Enrollment'].mean()
    datatomerge = datatomerge.reset_index()
    #merge enrollment data and star rating dataframe
    finaldf = pd.merge(newdf1,datatomerge,on=['Contract Number','Year'])
    #bucket the standardized enrollment data
    finaldf["Standardized Enrollment"] = finaldf["Standardized Enrollment"].apply(change_enrolled)
    #change enrollment column to int
    finaldf["Standardized Enrollment"] = finaldf["Standardized Enrollment"].astype(int)
    #return new dataframe
    return finaldf


In [ ]:
#Funtion to split numeric and non-numeric columns
#takes in a dataframe
#returns a dataframe with numeric and non-numeric columns
def split_numeric_nonnumeric_data(combodf4):
    #get list of columns
    cols = combodf4.keys().to_list()
    cols.append("Overall")
    #loop through first five columns
    for col in firstfive:
        cols.remove(col)
    #loop through last 2 columns
    for col in lasttwo:
        cols.remove(col)
    #loops througy columns in dataframe
    for col in cols:
        #add non-numeric columns
        combodf4[col+" Non-numeric"] = combodf4[col].apply(add_non_numeric_cols)
        #change old columns to numeric
        combodf4[col]=combodf4[col].apply(change_to_dec)
    newcols = []#placeholder for new column order
    #loop through columns
    for col in cols:
        #add numeric column
        newcols.append(col)
        #add non-numeric column
        newcols.append(col+" Non-numeric")
    finalcols =[]#placeholder for final column order
    #add columns to final columns
    for col in firstfive:
        finalcols.append(col)
    finalcols.append("Year")
    for col in newcols:
        finalcols.append(col)
    #change order of columns in dataframe
    newdf1 = combodf4[finalcols]
    return newdf1

In [ ]:
combodf = standardize_data()
combodf = clean_data(combodf)

In [ ]:
combodf = standardize_data()
combodf.to_csv("./CSVs/CombinedData.csv",header=True,index=False)

In [ ]:
combodf = clean_data(combodf)
combodf.to_csv("./CSVs/CombinedDataCleaned.csv",header=True,index=False)

C:\Users\samsi\AppData\Local\Temp\ipykernel_37080\2318447181.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combodf1 = combodf1.replace('\\s*No\\s*',False,regex=True)


In [ ]:
newdf1=split_numeric_nonnumeric_data(combodf)
newdf1.to_csv("./CSVs/CombinedDataNoEnrollment.csv",header=True,index=False)

In [ ]:
finaldf = combine_enrollment(newdf1)
finaldf.to_csv("./CSVs/FinalData.csv",header=True,index=False)